In [9]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from datetime import datetime


# 1. Connection Settings

In [2]:
# Load .env file
load_dotenv()

# Get the EC2 tracking server host from the environment variable
EC2_TRACKING_SERVER_HOST = os.getenv('EC2_TRACKING_SERVER_HOST')
EC2_ENDPOINT = f"http://{EC2_TRACKING_SERVER_HOST}:8000"

# Parameters for the RDS PostgreSQL instance
PG_HOST = os.getenv('PG_HOST')
PG_PORT = os.getenv('PG_PORT')
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

# Create the MySQL database connection string
db_url = f'postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}'

In [6]:
def ingest_rds(df, table_name, mode):
    engine = create_engine(db_url)
    connection = engine.connect()
    df.to_sql(table_name, engine, if_exists=mode, index=False)
    connection.close()

In [28]:
df = pd.DataFrame({
    'model_name': ['xgboost-8features-hpt', 'xgboost-8features-hpt-guster3'],
    'retrained_date': [datetime(2023,11,5,23,0,14) , datetime(2023,11,5,23,2,31)]  # Gets today's date
})

In [29]:
ingest_rds(df, 'table_update_rewa', 'append')

In [12]:
def select_from_rds(query):
    engine = create_engine(db_url)
    connection = engine.connect()
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [30]:
select_from_rds('select * from table_update_rewa')

,model_name,retrained_date
0,xgboost-8features-hpt,2023-11-05 23:00:14
1,xgboost-8features-hpt-guster3,2023-11-05 23:02:31
2,xgboost-8features-hpt,2023-11-05 23:00:14
3,xgboost-8features-hpt-guster3,2023-11-05 23:02:31


In [24]:
df_m = select_from_rds('select * from measurments_rewa')
df_m['Time'].max()

Timestamp('2023-11-07 20:30:00')

In [28]:
df = select_from_rds(query='select * from model_versions')